# Konsep Pembersihan Data

Pada notebook ini, kita akan membahas konsep dan teknik pembersihan data (data cleaning) yang merupakan tahap kritis dalam proses data science.

## Apa itu Pembersihan Data?

Pembersihan data adalah proses mendeteksi dan memperbaiki (atau menghapus) data yang rusak, tidak akurat, tidak lengkap, atau tidak relevan dalam dataset. Tujuannya adalah untuk meningkatkan kualitas data sebelum digunakan dalam analisis atau pemodelan.

## Mengapa Pembersihan Data Penting?

1. **Kualitas Analisis**
   - "Garbage In, Garbage Out" - hasil analisis sangat bergantung pada kualitas data input
   - Data yang bersih menghasilkan insight yang lebih akurat
   - Mengurangi bias dalam analisis

2. **Efisiensi Model**
   - Model machine learning bekerja lebih baik dengan data yang bersih
   - Mengurangi waktu training
   - Meningkatkan akurasi prediksi

3. **Keandalan Hasil**
   - Meningkatkan kepercayaan terhadap hasil analisis
   - Memudahkan interpretasi data
   - Mengurangi risiko kesalahan pengambilan keputusan

In [40]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Buat contoh dataset dengan berbagai masalah umum
np.random.seed(42)

# Buat data dummy dengan masalah-masalah umum
data = {
    'ID': range(1, 11),
    'Nama': ['John Doe', 'Jane Smith', 'Bob', np.nan, 'Alice Brown', 'Charlie', 'David Wilson', 'Eve', 'Frank', 'Grace'],
    'Usia': [25, -5, 30, 40, 35, 999, 45, 28, 33, 29],
    'Pendapatan': [5000000, 7000000, np.nan, 6500000, 8000000, 7500000, np.nan, 6000000, 7200000, 6800000],
    'Email': ['john@email.com', 'invalid-email', 'bob@email.com', 'alice@email', 'alice@email.com', 
              'charlie@email.com', 'david@email.com', 'eve@email.com', 'frank@email.com', 'grace@email.com']
}

# Buat DataFrame
df = pd.DataFrame(data)

# Tampilkan data awal
print("Dataset Awal:")
print(df)

# Tampilkan informasi dasar tentang dataset
print("\nInformasi Dataset:")
print(df.info())

# Tampilkan statistik deskriptif
print("\nStatistik Deskriptif:")
print(df.describe())

Dataset Awal:
   ID          Nama  Usia  Pendapatan              Email
0   1      John Doe    25   5000000.0     john@email.com
1   2    Jane Smith    -5   7000000.0      invalid-email
2   3           Bob    30         NaN      bob@email.com
3   4           NaN    40   6500000.0        alice@email
4   5   Alice Brown    35   8000000.0    alice@email.com
5   6       Charlie   999   7500000.0  charlie@email.com
6   7  David Wilson    45         NaN    david@email.com
7   8           Eve    28   6000000.0      eve@email.com
8   9         Frank    33   7200000.0    frank@email.com
9  10         Grace    29   6800000.0    grace@email.com

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          10 non-null     int64  
 1   Nama        9 non-null      object 
 2   Usia        10 non-null     int64  
 3   Pendapatan  8 non-null      

## Masalah Umum dalam Data

Dari dataset di atas, kita dapat mengidentifikasi beberapa masalah umum:

1. **Missing Values (Nilai yang Hilang)**
   - Nama yang kosong (NaN)
   - Pendapatan yang kosong (NaN)

2. **Invalid Values (Nilai Tidak Valid)**
   - Usia negatif (-5)
   - Usia yang tidak masuk akal (999)
   - Format email yang tidak valid

3. **Inconsistent Data (Data Tidak Konsisten)**
   - Format nama yang tidak konsisten (ada yang lengkap, ada yang hanya nama depan)
   - Format email yang tidak standar

Mari kita bahas cara menangani masalah-masalah ini satu per satu.

In [42]:
# 1. Menangani Missing Values

# Cek missing values
print("Jumlah Missing Values per Kolom:")
print(df.isnull().sum())

# Cara menangani missing values:

# a. Hapus baris dengan missing values
df_clean1 = df.dropna()
print("\nDataset setelah menghapus missing values:")
print(df_clean1)

# b. Isi missing values dengan nilai tertentu
df_clean2 = df.copy()
# Isi missing values di kolom Nama dengan "Unknown"
df_clean2['Nama'] = df_clean2['Nama'].fillna('Unknown')
# Isi missing values di kolom Pendapatan dengan mean
df_clean2['Pendapatan'] = df_clean2['Pendapatan'].fillna(df_clean2['Pendapatan'].mean())

print("\nDataset setelah mengisi missing values:")
print(df_clean2)

Jumlah Missing Values per Kolom:
ID            0
Nama          1
Usia          0
Pendapatan    2
Email         0
dtype: int64

Dataset setelah menghapus missing values:
   ID         Nama  Usia  Pendapatan              Email
0   1     John Doe    25   5000000.0     john@email.com
1   2   Jane Smith    -5   7000000.0      invalid-email
4   5  Alice Brown    35   8000000.0    alice@email.com
5   6      Charlie   999   7500000.0  charlie@email.com
7   8          Eve    28   6000000.0      eve@email.com
8   9        Frank    33   7200000.0    frank@email.com
9  10        Grace    29   6800000.0    grace@email.com

Dataset setelah mengisi missing values:
   ID          Nama  Usia  Pendapatan              Email
0   1      John Doe    25   5000000.0     john@email.com
1   2    Jane Smith    -5   7000000.0      invalid-email
2   3           Bob    30   6750000.0      bob@email.com
3   4       Unknown    40   6500000.0        alice@email
4   5   Alice Brown    35   8000000.0    alice@email.com


In [43]:
# 2. Menangani Invalid Values

# Fungsi untuk validasi usia
def clean_usia(usia):
    if usia < 0 or usia > 120:  # Asumsikan usia maksimal 120 tahun
        return np.nan
    return usia

# Fungsi untuk validasi email
import re
def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, email))

# Bersihkan data
df_clean3 = df_clean2.copy()

# Bersihkan usia
df_clean3['Usia'] = df_clean3['Usia'].apply(clean_usia)

# Bersihkan email
df_clean3['Email_Valid'] = df_clean3['Email'].apply(is_valid_email)

print("Dataset setelah validasi usia dan email:")
print(df_clean3)

# Tampilkan email yang tidak valid
print("\nEmail tidak valid:")
print(df_clean3[~df_clean3['Email_Valid']][['Nama', 'Email']])

Dataset setelah validasi usia dan email:
   ID          Nama  Usia  Pendapatan              Email  Email_Valid
0   1      John Doe  25.0   5000000.0     john@email.com         True
1   2    Jane Smith   NaN   7000000.0      invalid-email        False
2   3           Bob  30.0   6750000.0      bob@email.com         True
3   4       Unknown  40.0   6500000.0        alice@email        False
4   5   Alice Brown  35.0   8000000.0    alice@email.com         True
5   6       Charlie   NaN   7500000.0  charlie@email.com         True
6   7  David Wilson  45.0   6750000.0    david@email.com         True
7   8           Eve  28.0   6000000.0      eve@email.com         True
8   9         Frank  33.0   7200000.0    frank@email.com         True
9  10         Grace  29.0   6800000.0    grace@email.com         True

Email tidak valid:
         Nama          Email
1  Jane Smith  invalid-email
3     Unknown    alice@email


## Teknik Pembersihan Data Lanjutan

### 1. Menangani Outliers

Outlier adalah nilai yang sangat berbeda dari kebanyakan nilai dalam dataset. Ada beberapa metode untuk mendeteksi dan menangani outliers:

1. Metode IQR (Interquartile Range)
2. Z-score
3. Modified Z-score
4. Isolation Forest
5. Local Outlier Factor (LOF)

Mari kita implementasikan beberapa metode tersebut:

In [45]:
# Deteksi Outliers menggunakan IQR method
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

# Deteksi Outliers menggunakan Z-score
def detect_outliers_zscore(data, threshold=3):
    z_scores = (data - data.mean()) / data.std()
    return abs(z_scores) > threshold

# Aplikasikan kedua metode pada kolom numerik
numeric_columns = ['Usia', 'Pendapatan']
df_clean4 = df_clean3.copy()

print("Deteksi Outliers menggunakan IQR method:")
for col in numeric_columns:
    outliers = detect_outliers_iqr(df_clean4[col])
    print(f"\nOutliers di kolom {col}:")
    print(df_clean4[outliers][[col]])

print("\nDeteksi Outliers menggunakan Z-score:")
for col in numeric_columns:
    outliers = detect_outliers_zscore(df_clean4[col])
    print(f"\nOutliers di kolom {col}:")
    print(df_clean4[outliers][[col]])

Deteksi Outliers menggunakan IQR method:

Outliers di kolom Usia:
Empty DataFrame
Columns: [Usia]
Index: []

Outliers di kolom Pendapatan:
   Pendapatan
0   5000000.0

Deteksi Outliers menggunakan Z-score:

Outliers di kolom Usia:
Empty DataFrame
Columns: [Usia]
Index: []

Outliers di kolom Pendapatan:
Empty DataFrame
Columns: [Pendapatan]
Index: []


### 2. Standardisasi Format Data

Standardisasi format data adalah proses membuat data konsisten dalam hal format dan representasi. Beberapa contoh:

1. Format tanggal
2. Format nama
3. Format alamat
4. Format nomor telepon
5. Format mata uang

Mari kita implementasikan beberapa contoh standardisasi:

In [47]:
# Standardisasi format nama
def standardize_name(name):
    if pd.isna(name):
        return name
    # Capitalize each word
    return ' '.join(word.capitalize() for word in str(name).split())

# Standardisasi format mata uang
def format_currency(amount):
    if pd.isna(amount):
        return amount
    return f"Rp {amount:,.2f}"

# Terapkan standardisasi
df_clean5 = df_clean4.copy()
df_clean5['Nama'] = df_clean5['Nama'].apply(standardize_name)
df_clean5['Pendapatan_Formatted'] = df_clean5['Pendapatan'].apply(format_currency)

print("Dataset setelah standardisasi format:")
print(df_clean5[['Nama', 'Pendapatan_Formatted']])

# Contoh penambahan data tanggal
from datetime import datetime, timedelta

# Tambah kolom tanggal dengan format yang berbeda-beda
dates = [
    '2024-01-01',
    '01/02/2024',
    '03-Mar-2024',
    '2024.04.01',
    '05-05-2024',
    '06/06/2024',
    '2024-07-07',
    '08-Aug-2024',
    '09/09/2024',
    '2024.10.10'
]

df_clean5['Tanggal'] = dates

# Fungsi untuk standardisasi format tanggal
def standardize_date(date_str):
    try:
        # Coba beberapa format tanggal umum
        formats = [
            '%Y-%m-%d',
            '%d/%m/%Y',
            '%d-%b-%Y',
            '%Y.%m.%d',
            '%d-%m-%Y'
        ]
        
        for fmt in formats:
            try:
                return datetime.strptime(date_str, fmt).strftime('%Y-%m-%d')
            except ValueError:
                continue
                
        return None
    except:
        return None

# Terapkan standardisasi tanggal
df_clean5['Tanggal_Standardized'] = df_clean5['Tanggal'].apply(standardize_date)

print("\nDataset setelah standardisasi tanggal:")
print(df_clean5[['Tanggal', 'Tanggal_Standardized']])

Dataset setelah standardisasi format:
           Nama Pendapatan_Formatted
0      John Doe      Rp 5,000,000.00
1    Jane Smith      Rp 7,000,000.00
2           Bob      Rp 6,750,000.00
3       Unknown      Rp 6,500,000.00
4   Alice Brown      Rp 8,000,000.00
5       Charlie      Rp 7,500,000.00
6  David Wilson      Rp 6,750,000.00
7           Eve      Rp 6,000,000.00
8         Frank      Rp 7,200,000.00
9         Grace      Rp 6,800,000.00

Dataset setelah standardisasi tanggal:
       Tanggal Tanggal_Standardized
0   2024-01-01           2024-01-01
1   01/02/2024           2024-02-01
2  03-Mar-2024           2024-03-03
3   2024.04.01           2024-04-01
4   05-05-2024           2024-05-05
5   06/06/2024           2024-06-06
6   2024-07-07           2024-07-07
7  08-Aug-2024           2024-08-08
8   09/09/2024           2024-09-09
9   2024.10.10           2024-10-10


## Best Practices dalam Pembersihan Data

1. **Dokumentasi**
   - Catat semua perubahan yang dilakukan
   - Dokumentasikan alasan setiap perubahan
   - Simpan data asli

2. **Konsistensi**
   - Gunakan format yang konsisten
   - Terapkan aturan pembersihan yang sama untuk data serupa
   - Standardisasi nama kolom dan nilai

3. **Validasi**
   - Periksa hasil pembersihan
   - Validasi asumsi yang digunakan
   - Test dengan sampel data

4. **Automasi**
   - Buat fungsi untuk tugas pembersihan yang berulang
   - Gunakan pipeline data cleaning
   - Implementasi unit testing

## Latihan

1. Buat dataset dummy dengan berbagai masalah data
2. Implementasikan pembersihan data menggunakan berbagai metode
3. Buat pipeline pembersihan data
4. Dokumentasikan proses pembersihan

## Kesimpulan

Pembersihan data adalah proses yang kritis dalam data science yang memerlukan:
1. Pemahaman mendalam tentang data
2. Pengetahuan tentang berbagai teknik pembersihan
3. Kesabaran dan ketelitian
4. Dokumentasi yang baik

## Referensi

1. Pandas Documentation
2. Data Cleaning Best Practices
3. Python Data Science Handbook
4. Practical Statistics for Data Scientists